In [14]:
import json
import re

import requests
import pandas as pd
import awswrangler as wr
import utils as U
import time

In [23]:
import boto3
session=boto3
def athena_query(client, params):
    
    response = client.start_query_execution(
        QueryString=params["query"],
        QueryExecutionContext={
            'Database': params['database']
        },
        ResultConfiguration={
        'OutputLocation': 's3://tuzomldev/athena_results/'
    }
    )
    return response

def athena_to_s3(session, params, max_execution = 15):
    client = session.client('athena', region_name=params["region"])
    execution = athena_query(client, params)
    execution_id = execution['QueryExecutionId']
    print(execution_id)
    state = 'RUNNING'

    while (max_execution > 0 and state in ['RUNNING', 'QUEUED']):
        max_execution = max_execution - 1
        response = client.get_query_execution(QueryExecutionId = execution_id)

        if 'QueryExecution' in response and \
                'Status' in response['QueryExecution'] and \
                'State' in response['QueryExecution']['Status']:
            state = response['QueryExecution']['Status']['State']
            if state == 'FAILED':
                return False
            elif state == 'SUCCEEDED':
                s3_path = response['QueryExecution']['ResultConfiguration']['OutputLocation']
                filename = re.findall('.*\/(.*)', s3_path)[0]
                return filename
        time.sleep(1)
    
    return False

In [24]:
query="select * from ind_poi_data_v2_gold limit 10"
params={'database':'datasets_prep','query':query,"region":'us-east-2'}
athena_to_s3(session,params)

b0330f94-991a-4209-b7c0-c138248cad7e


'b0330f94-991a-4209-b7c0-c138248cad7e.csv'

In [28]:
wr.s3.read_csv("s3://tuzomldev/athena_results/b0330f94-991a-4209-b7c0-c138248cad7e.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 55: invalid start byte

In [30]:
import io
class QueryAthena:
    
    def __init__(self, query, database):
        self.database = database
        self.folder = 'athena_results'
        self.bucket = 'tuzomldev'
        self.s3_input = 's3://' + self.bucket + '/'+self.folder
        self.s3_output =  's3://' + self.bucket + '/' + self.folder
        self.region_name = 'us-east-2'
        self.query = query
        
    def load_conf(self, q):
        try:
            self.client = boto3.client('athena', region_name = self.region_name, )
            response = self.client.start_query_execution(
                QueryString = q,
                    QueryExecutionContext={
                    'Database': self.database
                    },
                    ResultConfiguration={
                    'OutputLocation': self.s3_output,
                    }
            )
            self.filename = response['QueryExecutionId']
            print('Execution ID: ' + response['QueryExecutionId'])
            return response

        except Exception as e:
            print(e)                
  
    def run_query(self):
        queries = [self.query]
        for q in queries:
            res = self.load_conf(q)
        try:              
            query_status = None
            while query_status == 'QUEUED' or query_status == 'RUNNING' or query_status is None:
                query_status = self.client.get_query_execution(QueryExecutionId=res["QueryExecutionId"])['QueryExecution']['Status']['State']
                print(query_status)
                if query_status == 'FAILED' or query_status == 'CANCELLED':
                    raise Exception('Athena query with the string "{}" failed or was cancelled'.format(self.query))
                time.sleep(1)
            print('Query "{}" finished.'.format(self.query))
            
            df = self.obtain_data()
            return df
            
        except Exception as e:
            print(e)      
            
    def obtain_data(self):
        try:
            self.resource = boto3.resource('s3', region_name = self.region_name, )

            response = self.resource \
            .Bucket(self.bucket) \
            .Object(key= self.folder+"/"+ self.filename + '.csv') \
            .get()
            
            return pd.read_csv(io.BytesIO(response['Body'].read()), encoding='utf8')   
        except Exception as e:
            print(e)  
           
        
if __name__ == "__main__":       
    query ="select * from ind_poi_data_v2_gold limit 10"
    qa = QueryAthena(query=query, database='datasets_prep')
    dataframe = qa.run_query()

Execution ID: 3b2d44f0-5c3f-44bd-903e-260045ec7149
RUNNING
SUCCEEDED
Query "select * from ind_poi_data_v2_gold limit 10" finished.


In [31]:
dataframe

,id,name,lat,lng,top_category,sub_categories,type,brand_name,brand_id,address,...,additional_data,geo_filter_1,geo_filter_2,service_options,oldest_review_timestamp,top_reviews,top_review_keywords,price_level,num_reviews_per_day,category
0,ChIJKS5efSIXrjsRPIF2QunVWGE,Ram Aviation India Office,13.022662,77.631340,education,[Aviation training institute],aviation_school,N_A,N_A,"2JFJ+3G8 Ram Aviation India Office, Chanakesha...",...,"{country=IN, city=Bengaluru, hexadecimal_cid=0...",123303312031,12330331203121,[],NaN,[],[],NaN,NaN,education_other
1,ChIJH4U5kZ09rjsRhGfl-KSHyek,Sri Prasanna Veeranjaneya Yoga Kendra,13.009082,77.546646,education,[School],N_A,N_A,N_A,2G5W+JMJ Sri Prasanna Veeranjaneya Yoga Kendra...,...,"{country=IN, city=Bengaluru, hexadecimal_cid=0...",123303312030,12330331203021,[],NaN,[],[],NaN,NaN,education_other
2,ChIJ2yAODuQRrjsRKEcFTa4ools,Narain Aviation,13.001672,77.661499,education,"[Aviation training institute, Charter school, ...",aviation_school,N_A,N_A,"No.2, M, Narain Aviation, 170, 2nd Main Rd, Ea...",...,"{country=IN, city=Bengaluru, hexadecimal_cid=0...",123303312031,12330331203132,[],NaN,[],[],NaN,NaN,education_other
3,ChIJoUN32uc9rjsRG92NnV3A_dQ,Vajra Art Creations,13.003798,77.545784,education,[Art school],N_A,N_A,N_A,"Vajra Art Creations, 362, 4th Main Rd, West of...",...,"{country=IN, city=Bengaluru, hexadecimal_cid=0...",123303312030,12330331203023,[],NaN,[],[],NaN,NaN,education_other
4,ChIJ4UmigeoXrjsREF8KBrbyjxI,Ken Pro Learning,13.025278,77.637192,education,"[Software Training Institute, Aviation trainin...",aviation_school,N_A,N_A,"Ken Pro Learning, #402 A, Soorya Plaza, 1 st F...",...,"{country=IN, city=Bengaluru, hexadecimal_cid=0...",123303312031,12330331203121,[],NaN,[],[],NaN,NaN,education_other
5,ChIJJUgSBcw9rjsRc6D1KLMLfg4,Bryan Edu Tech Pvt Ltd,12.997697,77.540154,education,[Education center],N_A,N_A,N_A,"Bryan Edu Tech Pvt Ltd, #8 1st floor behind li...",...,"{country=IN, city=Bengaluru, hexadecimal_cid=0...",123303312030,12330331203022,[],NaN,[],[],NaN,NaN,education_other
6,ChIJzYmg6S4WrjsRYB4NWcbVe8U,"Art for Heart,Drawing n Painting",13.006656,77.569542,education,[Art school],N_A,N_A,N_A,"Art for Heart,Drawing n Painting, 81,3rd main ...",...,"{country=IN, city=Bengaluru, hexadecimal_cid=0...",123303312030,12330331203030,[],NaN,[],[],NaN,NaN,education_other
7,ChIJ0WLkxs89rjsRLWeEpo2SxvM,The Creative Institute of Visual Art and Design,13.001902,77.550591,education,[Art school],N_A,N_A,N_A,The Creative Institute of Visual Art and Desig...,...,"{country=IN, city=Bengaluru, hexadecimal_cid=0...",123303312030,12330331203023,[],NaN,[],[],NaN,NaN,education_other
8,8bb5f927-b66c-41e3-b8ce-c44a1791be2a,KUNURI,23.916306,87.611518,transport,[N_A],N_A,N_A,N_A,NaN,...,"{code=KNRI, state=null, category=railway_stati...",123133300322,12313330032231,[],NaN,[],[],NaN,NaN,railway_station
9,6e966d12-af63-4a9c-9f62-c81c19baaf83,Kendriya Vidyalaya Minambakkam,12.989801,80.185895,company,"[Institutes - Educational, Training, Edtech]",Industry Top,N_A,N_A,"- Near Palavanthangal Railway Station, Minamb...",...,"{website=minambakkam.kvs.ac.in, scrape_url=htt...",123312212020,12331221202023,[],NaN,[],[],NaN,NaN,Govt Sector


In [27]:
session.client('athena').get_query_results(QueryExecutionId="b0330f94-991a-4209-b7c0-c138248cad7e")

{'UpdateCount': 0,
 'ResultSet': {'Rows': [{'Data': [{'VarCharValue': 'id'},
     {'VarCharValue': 'name'},
     {'VarCharValue': 'lat'},
     {'VarCharValue': 'lng'},
     {'VarCharValue': 'top_category'},
     {'VarCharValue': 'sub_categories'},
     {'VarCharValue': 'type'},
     {'VarCharValue': 'brand_name'},
     {'VarCharValue': 'brand_id'},
     {'VarCharValue': 'address'},
     {'VarCharValue': 'pincode'},
     {'VarCharValue': 'town_name'},
     {'VarCharValue': 'source'},
     {'VarCharValue': 'number_of_votes'},
     {'VarCharValue': 'rating'},
     {'VarCharValue': 'created_at'},
     {'VarCharValue': 'verified'},
     {'VarCharValue': 'last_verification_date'},
     {'VarCharValue': 'active'},
     {'VarCharValue': 'additional_data'},
     {'VarCharValue': 'geo_filter_1'},
     {'VarCharValue': 'geo_filter_2'},
     {'VarCharValue': 'service_options'},
     {'VarCharValue': 'oldest_review_timestamp'},
     {'VarCharValue': 'top_reviews'},
     {'VarCharValue': 'top_review

In [10]:
df = pd.read_csv(
    "/media/jyotiraditya/Ultra Touch/repos/SiteReports/data/common_data/blr_cluster_geom.csv")

In [11]:
df

,micro_market_clusters_id,cluster_name,locality,polygon,lat,lng,i20w,area,polygon_updated,i15d,i500md,i2kmd
0,0,"5th Cross Road, Koramangala 4th Block,Koramangala",Koramangala,POLYGON ((77.63591003417969 12.928566932678224...,12.931521,77.632015,"POLYGON ((77.628015 12.948565, 77.626369 12.94...",227159.994854,"POLYGON ((77.6363349851269 12.928819110832771,...","POLYGON ((77.641015 12.970552, 77.639015 12.96...","POLYGON ((77.630015 12.935083, 77.629015 12.93...","POLYGON ((77.629015 12.944764, 77.628015 12.94..."
1,1,"Hosur Road, Koramangala 5th Block,Koramangala",Koramangala,POLYGON ((77.61540985107422 12.931694984436035...,12.933954,77.613044,"POLYGON ((77.605044 12.95356, 77.603412 12.951...",278776.799324,POLYGON ((77.61540985107422 12.931694984436035...,"POLYGON ((77.619044 12.972156, 77.618314 12.97...","POLYGON ((77.615044 12.93511, 77.614488 12.934...","POLYGON ((77.621044 12.944, 77.619306 12.94369..."
2,2,"16th C Main Road, Koramangala 4th Block,Korama...",Koramangala,POLYGON ((77.62764739990234 12.933077812194824...,12.936409,77.626737,"POLYGON ((77.625737 12.952887, 77.623737 12.95...",320542.145753,POLYGON ((77.62764739990234 12.933077812194824...,"POLYGON ((77.640737 12.973428, 77.639737 12.97...","POLYGON ((77.627737 12.938249, 77.626047 12.93...","POLYGON ((77.620737 12.949375, 77.619737 12.94..."
3,3,"2nd Cross Road, Koramangala 5th Block,Koramangala",Koramangala,POLYGON ((77.62259674072266 12.931699752807615...,12.935219,77.619867,"POLYGON ((77.621867 12.952405, 77.618867 12.95...",324589.400644,POLYGON ((77.62259674072266 12.931699752807615...,"POLYGON ((77.621867 12.973405, 77.619867 12.97...","POLYGON ((77.620867 12.93855, 77.618867 12.938...","POLYGON ((77.619867 12.948596, 77.616867 12.94..."
4,4,"Whitefield Main Road, Palm Meadows,Whitefield",Whitefield,"POLYGON ((77.74502563476562 12.95724868774414,...",12.960090,77.746909,"POLYGON ((77.750909 12.979203, 77.749324 12.97...",145844.958247,POLYGON ((77.74560560950131 12.956376431776505...,"POLYGON ((77.757909 13.002159, 77.756909 13.00...","POLYGON ((77.748909 12.96431, 77.747909 12.964...","POLYGON ((77.750909 12.97772, 77.749868 12.977..."
...,...,...,...,...,...,...,...,...,...,...,...,...
81,122,"28th Cross Road, 7th Block Jaya Nagar,Jayanagar",Jayanagar,POLYGON ((77.57940673828125 12.928749084472656...,12.930212,77.577578,"POLYGON ((77.573578 12.949239, 77.570578 12.94...",77765.785154,"POLYGON ((77.5807397678116 12.929029007003413,...","POLYGON ((77.586578 12.969553, 77.585578 12.96...","POLYGON ((77.579578 12.933251, 77.578578 12.93...","POLYGON ((77.579578 12.945902, 77.577578 12.94..."
82,125,"Slv, Maheswari Nagar,T. Dasarahalli",T. Dasarahalli,"POLYGON ((77.5081138 13.0473777, 77.5065078735...",13.049413,77.507893,"POLYGON ((77.503893 13.069051, 77.501917 13.06...",49325.433149,POLYGON ((77.50954184387572 13.047029812806601...,"POLYGON ((77.504893 13.086599, 77.504522 13.08...","POLYGON ((77.508893 13.051654, 77.50793 13.051...","POLYGON ((77.505893 13.063746, 77.504711 13.06..."
83,129,"4th Main Road, Sadashiva Nagar,Armane Nagar",Armane Nagar,"POLYGON ((77.5812759399414 13.00473690032959, ...",13.006999,77.579610,"POLYGON ((77.58561 13.024219, 77.58461 13.0246...",119406.751266,POLYGON ((77.58243136966716 13.005005807284368...,"POLYGON ((77.59461 13.085288, 77.594603 13.085...","POLYGON ((77.58161 13.011006, 77.57861 13.0091...","POLYGON ((77.58361 13.021077, 77.58306 13.0209..."
84,131,"Outer Ring Road, Marathahalli village,Marathah...",Marathahalli,"POLYGON ((77.7015151977539 12.949820518493652,...",12.951217,77.699469,"POLYGON ((77.702469 12.969373, 77.701079 12.96...",98891.814473,POLYGON ((77.70270408382875 12.950281341395117...,"POLYGON ((77.683469 13.000504, 77.681469 13.00...","POLYGON ((77.700469 12.95603, 77.699499 12.955...","POLYGON ((77.701469 12.96909, 77.700073 12.967..."


In [12]:
wr.catalog.delete_table_if_exists(table='bng_high_streets',database='datasets_prep')
wr.s3.to_parquet(df, path="s3://tuzomldev/scraping_data/high_streets_data/bng_high_streets/",
                 table='bng_high_streets', database='datasets_prep', mode='append',dataset=True)

{'paths': ['s3://tuzomldev/scraping_data/high_streets_data/bng_high_streets/48bc636302c54b44a0c56c59521a77e2.snappy.parquet'],
 'partitions_values': {}}

In [1]:


class Dashboard:
    def __init__(self, report_id, id):
        self.report_id = report_id
        self.id = id
        self.project_info = U.get_project_info(self.report_id, self.id).to_dict(orient='records')[0]
        self.data = None
        self.data_dict = {}

    def update_cft(self):
        cft = self.data['data']['cft']['avg_cft']
        self.data_dict['avg_cost_for_two'] = cft

    def update_affluence(self):
        cft = self.data['data']['affluence']['affluence_index']
        self.data_dict['affluence'] = cft

    def update_location_score_weights_and_grouped_indexes(self):
        weights = json.dumps({
            'affluence_index': 0.19,
            'apartments_index': 0.19,
            'fashion_index': 0.19,
            'vibrancy': 0.14,
            'healthcare_index': 0.10,
            'company_index': 0.10,
            'malls_index': 0.05,
            'supermarket_index': 0.04,
        })
        self.data_dict['location_score_weights'] = weights
        indexes = json.dumps({'company_index': 3.89,
                              'connectivity_index': 4.62,
                              'education_index': 4.58,
                              'electronics_index': 4.78,
                              'entertainment_index': 5.0,
                              'fashion_index': 4.98,
                              'grocery_index': 4.98,
                              'healthcare_index': 4.82,
                              'home_decor_index': 4.56,
                              'leisure_index': 2.75,
                              'malls_index': 0.0,
                              'parks_index': 0.86,
                              'religious_index': 4.5,
                              'supermarket_index': 3.58,
                              'vibrancy': 4.01})
        self.data_dict['grouped_indexes'] = indexes

    def generate_report_data(self):
        eps = ["cft", "population", "companies", "demand_generator", "projects", "affluence", "income", "competition",
               "category_count", "property_price", "high_street", 'shopping_mall']
        all_data = {"data": {}}
        for ep in eps:
            url = f"http://127.0.0.1:8000/site_report/{self.report_id}/{self.id}/{ep}"
            req = requests.get(url)
            try:
                resp = req.json()
            except Exception as e:
                print(url, e)
                raise e
            data = resp['data']
            all_data['data'][ep] = data
        self.data = all_data

    def populate_single_cells(self):
        with open("dashboard_copy.json") as f:
            data = json.load(f)
        for i in data:
            try:
                i['values'] = [[self.get_dictionary_value(self.data, i['values'][0][0])]]
            except Exception as e:
                print(e, i['values'][0][0])
        # self.report_worksheet.batch_update(data)

    def insert_pois_ranked(self):
        a = pd.DataFrame(self.data['data']['category_count']['data'])['top_pois']
        from functools import reduce
        df = pd.DataFrame(reduce(lambda a, b: a + b, a.values.tolist())).sort_values(by='reviews_per_day',
                                                                                     ascending=False)
        df_ = df.to_dict(orient='records')[:]
        pois = df_
        df = df[df['brand_id'] != 'N_A'].to_dict(orient='records')[:]
        df = json.dumps(df)
        self.data_dict['top_brands'] = df
        return pois

    def update_category_count(self):
        buff = self.data['data']['category_count']
        self.data_dict['pois'] = json.dumps(buff)

    def update_high_streets(self):
        high_streets = self.data['data']['high_street']
        high_streets = json.dumps(high_streets)
        self.data_dict['high_streets'] = high_streets

    def update_competition(self):
        comp = self.data['data']['competition']
        comp = json.dumps(comp)
        self.data_dict['competition'] = comp

    def update_malls(self):
        comp = self.data['data']['shopping_mall']
        comp = json.dumps(comp)
        self.data_dict['shopping_malls'] = comp

    def update_household_distribution(self):
        comp = self.data['data']['income']
        comp = json.dumps(comp)
        self.data_dict['household_distribution'] = comp

    def update_property_price(self):
        # comp = self.data['data']['property_price']['price']
        # comp = json.dumps(comp)
        # self.data_dict['property_prices'] = comp
        comp = self.data['data']['property_price']
        # comp[0]['top_pois'] = comp[0]['top_pois'][:]
        comp = json.dumps(comp)
        self.data_dict['apartments'] = comp

    def update_projects(self):
        p = self.data['data']['projects']
        p['projects'] = p['projects'][:]
        self.data_dict['projects'] = json.dumps(p)

    def update_companies(self):
        companies = self.data['data']['companies']
        demand_generators = self.data['data']['demand_generator']
        demand_generators['companies'] = companies
        self.data_dict['demand_generators'] = json.dumps(demand_generators)

    def get_revenue_score(self):
        self.data_dict['revenue_score'] = 81.95

    def population_chart(self):
        pop = self.data['data']['population']
        pop = json.dumps(pop)
        self.data_dict['population'] = pop

    @staticmethod
    def get_dictionary_value(d, keys):
        keys = keys.split(".")
        for i in keys:
            d = d.get(i)
        return d

    def generate_report(self):
        self.generate_report_data()
        # self.populate_single_cells()
        self.update_cft()
        # self.update_location_score_weights_and_grouped_indexes()
        self.update_affluence()
        self.insert_pois_ranked()
        self.update_household_distribution()
        self.update_property_price()
        self.update_projects()
        self.update_malls()
        self.population_chart()
        self.update_competition()
        self.update_category_count()
        self.update_high_streets()
        # self.get_revenue_score()
        self.update_companies()


In [11]:
import utils as U
import importlib

importlib.reload(U)
all_reports = U.get_project_info().tail(4)

In [12]:
all_reports

,report_id,id,site_name,lat,lng,geometry,location,catchment_type,top_brands,pois,...,projects_counts,created_at,location_score_weights,grouped_indexes,demand_generators,city_lat,city_lng,distance_from_city_center,orientation_from_city_center,competitors_domains
16,f9b43dc5-f8fe-4cb0-b91a-54cbf23de536,129342_777438,"Orion Utopia - Sh35,Varthur",12.934247,77.743869,"POLYGON ((77.751869 12.98133, 77.749635 12.979...","Sh35,Varthur",i15mind,"[{""id"": ""ChIJA7wj8BMSrjsROKK4nf2PcCA"", ""name"":...","{""count"": 376, ""avg_number_of_reviews_per_day""...",...,None,1707378645,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 406, ""college"": 51, ""hospital"": 147...",12.976794,77.590082,NaN,SE,[]
17,ba9999e5-b9f2-4eb0-8b71-eec0c9af1930,129342_777438,"Orion Utopia - Sh35,Varthur",12.934247,77.743869,"POLYGON ((77.745869 12.937926, 77.743614 12.93...","Sh35,Varthur",i500mtd,"[{""id"": ""ChIJr8OCGv8TrjsRbG5HtKs6Pm8"", ""name"":...","{""count"": 11, ""avg_number_of_reviews_per_day"":...",...,None,1707378647,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 14, ""hospital"": 5, ""pharmacy"": 3, ""...",12.976794,77.590082,NaN,SE,[]
18,59237f70-29b4-463c-a869-8323df9f54ca,128050_776996,"Orion Valencia - Hosur Road, Keerthi Layout,Su...",12.805019,77.699684,"POLYGON ((77.647684 12.877109, 77.646743 12.87...","Hosur Road, Keerthi Layout,Suryanagar",i15mind,"[{""id"": ""ChIJba_U8IxsrjsRK4d0fwEn3KA"", ""name"":...","{""count"": 318, ""avg_number_of_reviews_per_day""...",...,None,1707378648,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 504, ""college"": 137, ""hospital"": 28...",12.976794,77.590082,NaN,SE,[]
19,df93c3fa-d9ac-46ad-9b8a-d423fb715ae6,128050_776996,"Orion Valencia - Hosur Road, Keerthi Layout,Su...",12.805019,77.699684,"POLYGON ((77.698684 12.808065, 77.697772 12.80...","Hosur Road, Keerthi Layout,Suryanagar",i500mtd,"[{""id"": ""ChIJ7VcOZjNsrjsR88oQJA5Q2xc"", ""name"":...","{""count"": 7, ""avg_number_of_reviews_per_day"": ...",...,None,1707378649,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 5, ""hospital"": 11, ""pharmacy"": 4, ""...",12.976794,77.590082,NaN,SE,[]


In [4]:
all_reports = all_reports.query('catchment_type!="i1000mtb" and id not in ("129065_775925","130636_775869")')  #.tail(4)

In [4]:
query = '''UPDATE ind_site_reports_v2
SET
    top_brands = :top_brands,
    pois = :pois,
    projects = :projects,
    apartments = :apartments,
    household_distribution = :household_distribution,
    competition = :competition,
    population = :population,
    affluence = :affluence,
    avg_cost_for_two = :avg_cost_for_two,
    high_streets = :high_streets,
    shopping_malls = :shopping_malls,
    demand_generators = :demand_generators
WHERE report_id = :report_id and id=:id;
'''

In [8]:
if __name__ == '__main__':
    con, cur = U.connect_to_db()

    for proj in all_reports.to_dict(orient='records')[:]:
        rep = Dashboard(report_id=proj['report_id'], id=proj['id'])
        rep.generate_report()
        query_params = rep.data_dict
        query_params['report_id'] = proj['report_id']
        query_params['id'] = proj['id']
        cur.execute(query, query_params)
        con.commit()
    cur.close()
    con.close()


In [22]:
df = U.get_all_reports().tail(4)
df

,report_id,id,site_name,lat,lng,geometry,location,catchment_type,top_brands,pois,...,projects_counts,created_at,location_score_weights,grouped_indexes,demand_generators,city_lat,city_lng,distance_from_city_center,orientation_from_city_center,competitors_domains
16,f9b43dc5-f8fe-4cb0-b91a-54cbf23de536,129342_777438,"Orion Utopia - Sh35,Varthur",12.934247,77.743869,"POLYGON ((77.751869 12.98133, 77.749635 12.979...","Sh35,Varthur",i15mind,"[{""id"": ""ChIJA7wj8BMSrjsROKK4nf2PcCA"", ""name"":...","{""count"": 376, ""avg_number_of_reviews_per_day""...",...,None,1707378645,"{""affluence_index"": 0.237, ""apartments_index"":...","{""affluence_index"": 3.39, ""apartments_index"": ...","{""clinic"": 406, ""college"": 51, ""hospital"": 147...",12.976794,77.590082,17.337685,SE,[]
17,ba9999e5-b9f2-4eb0-8b71-eec0c9af1930,129342_777438,"Orion Utopia - Sh35,Varthur",12.934247,77.743869,"POLYGON ((77.745869 12.937926, 77.743614 12.93...","Sh35,Varthur",i500mtd,"[{""id"": ""ChIJr8OCGv8TrjsRbG5HtKs6Pm8"", ""name"":...","{""count"": 11, ""avg_number_of_reviews_per_day"":...",...,None,1707378647,"{""affluence_index"": 0.237, ""apartments_index"":...","{""affluence_index"": 3.39, ""apartments_index"": ...","{""clinic"": 14, ""hospital"": 5, ""pharmacy"": 3, ""...",12.976794,77.590082,17.337685,SE,[]
18,59237f70-29b4-463c-a869-8323df9f54ca,128050_776996,"Orion Valencia - Hosur Road, Keerthi Layout,Su...",12.805019,77.699684,"POLYGON ((77.647684 12.877109, 77.646743 12.87...","Hosur Road, Keerthi Layout,Suryanagar",i15mind,"[{""id"": ""ChIJba_U8IxsrjsRK4d0fwEn3KA"", ""name"":...","{""count"": 318, ""avg_number_of_reviews_per_day""...",...,None,1707378648,"{""affluence_index"": 0.237, ""apartments_index"":...","{""affluence_index"": 2.67, ""apartments_index"": ...","{""clinic"": 504, ""college"": 137, ""hospital"": 28...",12.976794,77.590082,22.419331,SE,[]
19,df93c3fa-d9ac-46ad-9b8a-d423fb715ae6,128050_776996,"Orion Valencia - Hosur Road, Keerthi Layout,Su...",12.805019,77.699684,"POLYGON ((77.698684 12.808065, 77.697772 12.80...","Hosur Road, Keerthi Layout,Suryanagar",i500mtd,"[{""id"": ""ChIJ7VcOZjNsrjsR88oQJA5Q2xc"", ""name"":...","{""count"": 7, ""avg_number_of_reviews_per_day"": ...",...,None,1707378649,"{""affluence_index"": 0.237, ""apartments_index"":...","{""affluence_index"": 2.67, ""apartments_index"": ...","{""clinic"": 5, ""hospital"": 11, ""pharmacy"": 4, ""...",12.976794,77.590082,22.419331,SE,[]


In [34]:
df = df.query('catchment_type!="i1000mtb" and id not in ("129065_775925","130636_775869")')

In [23]:
df = df.drop(columns=['poi_counts', 'projects_counts', 'median_price'])

In [50]:
wr.catalog.delete_table_if_exists(database='datasets_prep', table='ind_site_reports_data_v2')

True

In [24]:
wr.s3.to_parquet(df, path="s3://tuzomldev/site_reports/", table="ind_site_reports_data_v2", database='datasets_prep',
                 dataset=True, mode='append',
                 dtype={'report_id': 'string', 'id': 'string', 'site_name': 'string', 'lat': 'double',
                        'lng': 'double', 'catchment_type': 'string', 'top_brands': 'string', 'pois': 'string',
                        'projects': 'string', 'apartments': 'string', 'median_price': 'string',
                        'household_distribution': 'string', 'competition': 'string', 'population': 'string',
                        'affluence': 'double', 'avg_cost_for_two': 'double', 'revenue_score': 'double',
                        'high_streets': 'string', 'shopping_malls': 'string', 'poi_counts': 'string',
                        'projects_counts': 'string', 'created_at': 'bigint', "companies": "string"})

{'paths': ['s3://tuzomldev/site_reports/69f4e773772e4263886c65d8bec6a45d.snappy.parquet'],
 'partitions_values': {}}

In [12]:
df.shape

,report_id,id,site_name,lat,lng,geometry,location,catchment_type,top_brands,pois,...,population,affluence,avg_cost_for_two,revenue_score,high_streets,shopping_malls,created_at,location_score_weights,grouped_indexes,demand_generators
0,1f78f3ac-d799-470b-9738-2c4da5053cee,130110_775547,"Orion Mall - Dr. Rajkumar Road, Rajaji Nagar I...",13.011027,77.554736,"POLYGON ((77.494736 13.050067, 77.493202 13.05...","Dr. Rajkumar Road, Rajaji Nagar Industrial Sub...",i15mind,"[{""id"": ""ChIJe9qj3xs9rjsRDN6CamLC-Co"", ""name"":...","{""count"": 1083, ""avg_number_of_reviews_per_day...",...,"{""age_0_19"": 278764.7038758153, ""age_20_34"": 2...",3.675222,460.911885,81.950000,"[{""cluster_name"": ""1st Main Club Road Near Foo...","{""count"": 8, ""pois"": [{""id"": ""ChIJoRKfACMWrjsR...",1706512914,"{""companies"": 0.11, ""vibrancy"": 0.16, ""transpo...","{""company_index"": 3.89, ""connectivity_index"": ...","{""clinic"": 2093, ""college"": 413, ""hospital"": 6..."
1,2f78f3ac-d799-470b-9738-2c4da5053cee,130110_775547,"Orion Mall - Dr. Rajkumar Road, Rajaji Nagar I...",13.011027,77.554736,POLYGON ((77.55064773796063 13.010557348159388...,"Dr. Rajkumar Road, Rajaji Nagar Industrial Sub...",i500mtb,"[{""id"": ""ChIJe9qj3xs9rjsRDN6CamLC-Co"", ""name"":...","{""count"": 131, ""avg_number_of_reviews_per_day""...",...,"{""age_0_19"": 4275.960897406801, ""age_20_34"": 4...",4.127932,838.805970,81.950000,"[{""cluster_name"": ""Dr. Rajkumar Road, Rajaji N...","{""count"": 1, ""pois"": [{""id"": ""ChIJCUMusVM9rjsR...",1706512950,"{""companies"": 0.11, ""vibrancy"": 0.16, ""transpo...","{""company_index"": 3.89, ""connectivity_index"": ...","{""clinic"": 25, ""college"": 7, ""hospital"": 57, ""..."
2,374c8386-5837-43b8-8409-c456849263e7,130009_776325,Orion Avenue - Second Cross N. Thyagaraju Layo...,13.000957,77.632571,POLYGON ((77.62847982126846 13.000488913937247...,"Second Cross N. Thyagaraju Layout, Sathya Naga...",i500mtb,"[{""id"": ""ChIJAfIetiRpUjoRKDXrDBoNP-0"", ""name"":...","{""count"": 24, ""avg_number_of_reviews_per_day"":...",...,"{""age_0_19"": 4633.231310760981, ""age_20_34"": 4...",3.616629,500.000000,82.470000,[],"{""count"": 0, ""pois"": []}",1706684037,"{""companies"": 0.11, ""vibrancy"": 0.16, ""transpo...","{'company_index': 4.01, 'connectivity_index': ...","{""clinic"": 27, ""hospital"": 5, ""pharmacy"": 8, ""..."
3,2ef72172-e65c-47df-9f06-2a98c9f1fa94,130009_776325,Orion Avenue - Second Cross N. Thyagaraju Layo...,13.000957,77.632571,"POLYGON ((77.644571 13.032047, 77.643571 13.03...","Second Cross N. Thyagaraju Layout, Sathya Naga...",i15mind,"[{""id"": ""18767303"", ""name"": ""Imperio Restauran...","{""count"": 674, ""avg_number_of_reviews_per_day""...",...,"{""age_0_19"": 164433.15741358572, ""age_20_34"": ...",3.506014,498.559078,82.470000,"[{""cluster_name"": ""8th Main 3rd Phase Peenya I...","{""count"": 0, ""pois"": []}",1706684064,"{""companies"": 0.11, ""vibrancy"": 0.16, ""transpo...","{'company_index': 4.01, 'connectivity_index': ...","{""clinic"": 1313, ""college"": 135, ""hospital"": 3..."
4,5fca8372-db67-4e32-9c34-17818bca7c19,130552_777638,"Orion Uptown - Old Madras Road, Kattanallur,Sa...",13.055260,77.763894,"POLYGON ((77.773894 13.116797, 77.772949 13.11...","Old Madras Road, Kattanallur,Sannatammanahalli",i15mind,"[{""id"": ""ChIJdUCMjRcPrjsRjZ7tgReBR3M"", ""name"":...","{""count"": 85, ""avg_number_of_reviews_per_day"":...",...,"{""age_0_19"": 37307.81912734546, ""age_20_34"": 3...",3.134148,535.593220,40.173600,[],"{""count"": 2, ""pois"": [{""id"": ""ChIJfU4i1c0PrjsR...",1706691413,"{""companies"": 0.11, ""vibrancy"": 0.16, ""transpo...","{'company_index': 2.15, 'connectivity_index': ...","{""clinic"": 187, ""college"": 67, ""hospital"": 89,..."
5,ed374c4c-6dc3-473b-867e-ff707e347939,130552_777638,"Orion Uptown - Old Madras Road, Kattanallur,Sa...",13.055260,77.763894,"POLYGON ((77.75979674534983 13.05479248128713,...","Old Madras Road, Kattanallur,Sannatammanahalli",i500mtb,"[{""id"": ""ChIJu_szAwoPrjsR

In [9]:
import importlib

In [13]:
import revenue_score as rev

importlib.reload(rev)


<module 'revenue_score' from '/home/jyotiraditya/PycharmProjects/SiteReports/revenue_score.py'>

In [14]:
all_reports

,report_id,id,site_name,lat,lng,geometry,location,catchment_type,top_brands,pois,...,projects_counts,created_at,location_score_weights,grouped_indexes,demand_generators,city_lat,city_lng,distance_from_city_center,orientation_from_city_center,competitors_domains
16,f9b43dc5-f8fe-4cb0-b91a-54cbf23de536,129342_777438,"Orion Utopia - Sh35,Varthur",12.934247,77.743869,"POLYGON ((77.751869 12.98133, 77.749635 12.979...","Sh35,Varthur",i15mind,"[{""id"": ""ChIJA7wj8BMSrjsROKK4nf2PcCA"", ""name"":...","{""count"": 376, ""avg_number_of_reviews_per_day""...",...,None,1707378645,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 406, ""college"": 51, ""hospital"": 147...",12.976794,77.590082,NaN,SE,[]
17,ba9999e5-b9f2-4eb0-8b71-eec0c9af1930,129342_777438,"Orion Utopia - Sh35,Varthur",12.934247,77.743869,"POLYGON ((77.745869 12.937926, 77.743614 12.93...","Sh35,Varthur",i500mtd,"[{""id"": ""ChIJr8OCGv8TrjsRbG5HtKs6Pm8"", ""name"":...","{""count"": 11, ""avg_number_of_reviews_per_day"":...",...,None,1707378647,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 14, ""hospital"": 5, ""pharmacy"": 3, ""...",12.976794,77.590082,NaN,SE,[]
18,59237f70-29b4-463c-a869-8323df9f54ca,128050_776996,"Orion Valencia - Hosur Road, Keerthi Layout,Su...",12.805019,77.699684,"POLYGON ((77.647684 12.877109, 77.646743 12.87...","Hosur Road, Keerthi Layout,Suryanagar",i15mind,"[{""id"": ""ChIJba_U8IxsrjsRK4d0fwEn3KA"", ""name"":...","{""count"": 318, ""avg_number_of_reviews_per_day""...",...,None,1707378648,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 504, ""college"": 137, ""hospital"": 28...",12.976794,77.590082,NaN,SE,[]
19,df93c3fa-d9ac-46ad-9b8a-d423fb715ae6,128050_776996,"Orion Valencia - Hosur Road, Keerthi Layout,Su...",12.805019,77.699684,"POLYGON ((77.698684 12.808065, 77.697772 12.80...","Hosur Road, Keerthi Layout,Suryanagar",i500mtd,"[{""id"": ""ChIJ7VcOZjNsrjsR88oQJA5Q2xc"", ""name"":...","{""count"": 7, ""avg_number_of_reviews_per_day"": ...",...,None,1707378649,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 5, ""hospital"": 11, ""pharmacy"": 4, ""...",12.976794,77.590082,NaN,SE,[]


In [16]:
query = '''UPDATE ind_site_reports_v2
SET
    revenue_score=:revenue_score,
    grouped_indexes=:grouped_indexes,
    location_score_weights=:location_score_weights,
    affluence=:affluence
WHERE  id=:id and report_id=:report_id;
'''
# weights = json.dumps({
#     'affluence_index': 0.19,
#     'apartments_index': 0.19,
#     'fashion_index': 0.19,
#     'vibrancy': 0.14,
#     'healthcare_index': 0.10,
#     'company_index': 0.10,
#     'malls_index': 0.05,
#     'supermarket_index': 0.04,
# })
if __name__ == '__main__':
    con, cur = U.connect_to_db()
    dd = []
    for proj in all_reports.to_dict('records')[:]:
        print(proj['catchment_type'])
        pattern = re.compile("i(?P<cost>\d+)(?P<cost_type>[a-z]+)(?P<travel_mode>[d|b])$")
        a = pattern.search(proj['catchment_type']).groupdict()
        cost_types = {"mt": 'distance', 'min': "time"}
        travel_modes = {"d": 'driving', 'b': "buffer"}
        competitors = json.loads(proj['competitors_domains'])
        primary_comp = list(filter(lambda x: x['competitor_type'] == 'primary_competitor', competitors))
        anchor_comp = list(filter(lambda x: x['competitor_type'] == 'anchor_competitor', competitors))

        weight_score = json.loads(proj['location_score_weights'])
        try:
            del weight_score['healthcare_index']
            del weight_score['company_index']
        except:
            pass
        weight_score = {k: round(v / sum(weight_score.values()), 3) for k, v in weight_score.items()}
        if primary_comp:
            primary_comp = primary_comp[0]['filter']
            primary_comp = list(filter(lambda x: "'" not in x, primary_comp))
        if anchor_comp:
            anchor_comp = anchor_comp[0]['filter']
            anchor_comp = list(filter(lambda x: "'" not in x, anchor_comp))
        cost_type = cost_types[a['cost_type']]
        travel_mode = travel_modes[a['travel_mode']]
        cost = int(a['cost'])
        site_id = proj['id']
        report_id = proj['report_id']
        lat = proj['lat']
        lng = proj['lng']
        print(report_id, site_id, proj['catchment_type'], weight_score)
        print(travel_mode, cost_type, cost, primary_comp, anchor_comp)
        try:
            revenue_score, grouped_indexs = rev.generate_revenue_score(lat, lng, travel_mode, cost_type, cost,
                                                                       primary_comp, anchor_comp, weight_score)
            affluence = grouped_indexs['affluence_index']
            grouped_indexs = json.dumps(grouped_indexs)
            query_params = dict(revenue_score=revenue_score, grouped_indexes=grouped_indexs, id=site_id,
                                report_id=report_id, location_score_weights=json.dumps(weight_score),
                                affluence=affluence)
            dd.append(query_params)
            cur.execute(query, query_params)
            con.commit()
        except Exception as e:
            print(e)
    cur.close()
    con.close()


i15mind
f9b43dc5-f8fe-4cb0-b91a-54cbf23de536 129342_777438 i15mind {'affluence_index': 0.237, 'apartments_index': 0.237, 'fashion_index': 0.237, 'vibrancy': 0.175, 'malls_index': 0.062, 'supermarket_index': 0.05}
driving time 15 [] []


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

int() argument must be a string, a bytes-like object or a real number, not 'NoneType'


In [17]:
from geopy.distance import geodesic, great_circle

In [18]:
def get_direction_from_angle(angle):
    if -10 <= angle <= 10:
        return 'East'
    elif 10 < angle < 80:
        return 'North-East'
    elif 80 <= angle <= 110:
        return 'North'
    elif 110 < angle < 170:
        return 'North-West'
    elif 170 <= angle <= 180:
        return 'West'
    elif -10 > angle > -80:
        return 'South-East'
    elif -80 >= angle == -110:
        return 'South'
    elif -110 > angle > -170:
        return 'South-West'
    elif -170 >= angle >= -180:
        return 'West'
    else:
        raise ValueError('Angle Not Valid')


def get_relative_direction(x1, y1, x2, y2):
    import math
    if (x1, y1) == (x2, y2):
        return 'Same'
    # Calculate the angle between the two points in radians
    angle = math.atan2(y2 - y1, x2 - x1)

    # Convert the angle from radians to degrees
    angle_degrees = math.degrees(angle)
    # print(angle_degrees)
    # Convert the angle to a positive value between 0 and 360 degrees
    return get_direction_from_angle(angle_degrees)

In [19]:
def direction_lookup(destination_x, destination_y, origin_x, origin_y):
    import math
    deg2rad = math.pi / 180
    destination_x = destination_x / deg2rad
    destination_y = destination_y / deg2rad
    origin_x = origin_x / deg2rad
    origin_y = origin_y / deg2rad
    deltaX = destination_x - origin_x
    deltaY = destination_y - origin_y
    degrees_temp = math.atan2(deltaX, deltaY) / math.pi * 180
    if degrees_temp < 0:
        degrees_final = 360 + degrees_temp
    else:
        degrees_final = degrees_temp
    compass_brackets = ["N", "NE", "E", "SE", "S", "SW", "W", "NW", "N"]
    compass_lookup = round(degrees_final / 45)

    return compass_brackets[compass_lookup]

In [20]:
all_reports

,report_id,id,site_name,lat,lng,geometry,location,catchment_type,top_brands,pois,...,projects_counts,created_at,location_score_weights,grouped_indexes,demand_generators,city_lat,city_lng,distance_from_city_center,orientation_from_city_center,competitors_domains
16,f9b43dc5-f8fe-4cb0-b91a-54cbf23de536,129342_777438,"Orion Utopia - Sh35,Varthur",12.934247,77.743869,"POLYGON ((77.751869 12.98133, 77.749635 12.979...","Sh35,Varthur",i15mind,"[{""id"": ""ChIJA7wj8BMSrjsROKK4nf2PcCA"", ""name"":...","{""count"": 376, ""avg_number_of_reviews_per_day""...",...,None,1707378645,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 406, ""college"": 51, ""hospital"": 147...",12.976794,77.590082,NaN,SE,[]
17,ba9999e5-b9f2-4eb0-8b71-eec0c9af1930,129342_777438,"Orion Utopia - Sh35,Varthur",12.934247,77.743869,"POLYGON ((77.745869 12.937926, 77.743614 12.93...","Sh35,Varthur",i500mtd,"[{""id"": ""ChIJr8OCGv8TrjsRbG5HtKs6Pm8"", ""name"":...","{""count"": 11, ""avg_number_of_reviews_per_day"":...",...,None,1707378647,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 14, ""hospital"": 5, ""pharmacy"": 3, ""...",12.976794,77.590082,NaN,SE,[]
18,59237f70-29b4-463c-a869-8323df9f54ca,128050_776996,"Orion Valencia - Hosur Road, Keerthi Layout,Su...",12.805019,77.699684,"POLYGON ((77.647684 12.877109, 77.646743 12.87...","Hosur Road, Keerthi Layout,Suryanagar",i15mind,"[{""id"": ""ChIJba_U8IxsrjsRK4d0fwEn3KA"", ""name"":...","{""count"": 318, ""avg_number_of_reviews_per_day""...",...,None,1707378648,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 504, ""college"": 137, ""hospital"": 28...",12.976794,77.590082,NaN,SE,[]
19,df93c3fa-d9ac-46ad-9b8a-d423fb715ae6,128050_776996,"Orion Valencia - Hosur Road, Keerthi Layout,Su...",12.805019,77.699684,"POLYGON ((77.698684 12.808065, 77.697772 12.80...","Hosur Road, Keerthi Layout,Suryanagar",i500mtd,"[{""id"": ""ChIJ7VcOZjNsrjsR88oQJA5Q2xc"", ""name"":...","{""count"": 7, ""avg_number_of_reviews_per_day"": ...",...,None,1707378649,"{""affluence_index"": 0.237, ""apartments_index"":...",None,"{""clinic"": 5, ""hospital"": 11, ""pharmacy"": 4, ""...",12.976794,77.590082,NaN,SE,[]


In [21]:
query = '''UPDATE ind_site_reports_v2
SET
    distance_from_city_center=:distance_from_city_center,
    orientation_from_city_center=:orientation_from_city_center
WHERE  id=:id;
'''

if __name__ == '__main__':
    con, cur = U.connect_to_db()
    a = []
    for proj in all_reports.drop_duplicates(subset=['id'])[['id', 'lat', 'lng', 'city_lat', 'city_lng']].to_dict(
            'records'):
        site_id = proj['id']
        lat = proj['lat']
        lng = proj['lng']
        city_lat = proj['city_lat']
        city_lng = proj['city_lng']
        dist = geodesic((lat, lng), (city_lat, city_lng)).km
        # dir = direction_lookup(city_lat, city_lng,lat, lng)
        dir = direction_lookup(lat, lng, city_lat, city_lng, )
        query_params = dict(distance_from_city_center=dist, orientation_from_city_center=dir, id=site_id)
        cur.execute(query, query_params)
        a.append(query_params)
        con.commit()
    cur.close()
    con.close()

In [47]:
pd.DataFrame(a)

,distance_from_city_center,orientation_from_city_center,id
0,5.399600,NW,130110_775547
1,5.330382,SW,130009_776325
2,20.753626,SW,130552_777638
3,10.156619,W,130631_776205
4,9.970600,E,128873_775969
